### Load Pretrained Model
Load a pretrained Mamba Model that is compatible with Transformers Library

In [ ]:
from modeling_mamba import MambaForCausalLM
from transformers import AutoTokenizer

model = MambaForCausalLM.from_pretrained("Q-bert/Mamba-130M")
tokenizer = AutoTokenizer.from_pretrained("Q-bert/Mamba-130M")

text = "Hi"

input_ids = tokenizer.encode(text, return_tensors="pt")

output = model.generate(input_ids, max_length=20, num_beams=5, no_repeat_ngram_size=2)

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

print(generated_text)

In [ ]:
print([(n, type(m)) for n, m in model.named_modules()])

In [ ]:
print(type(model))


def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )


print("plain", print_trainable_parameters(model))

### Add LoRA adapters
1. Identify a particular layer in the Mamba and add an LoRA layer there
2. At this time, is only layer to verify if the code works


In [ ]:
from peft import LoraConfig, TaskType

target_modules = ["model.layers.3.x_proj"]

config = LoraConfig(target_modules=target_modules, task_type="CAUSAL_LM")

In [ ]:
from peft import get_peft_model

model = get_peft_model(model, config)
model.print_trainable_parameters()

In [ ]:
model.save_pretrained("wts")

### Merge the adpater into the Model
merge the adapter back to the model, so the merged model will have exactly the same architecture
except with the weights modified

In [ ]:
from peft import PeftConfig, PeftModel

adapter_path = "./wts/"
adapter_config = PeftConfig.from_pretrained(adapter_path)
base_model = MambaForCausalLM.from_pretrained("Q-bert/Mamba-130M")
adapted_model = PeftModel.from_pretrained(base_model, adapter_path)

In [ ]:
m = adapted_model.merge_and_unload()

In [ ]:
text = "Hi, Mamba is a"

input_ids = tokenizer.encode(text, return_tensors="pt")

output = m.generate(input_ids, max_length=20, num_beams=5, no_repeat_ngram_size=2)

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

print(generated_text)

In [ ]:
print("base mamba", print_trainable_parameters(base_model))
print("lora mamba", print_trainable_parameters(model))
print("merged mamba", print_trainable_parameters(m))

In [ ]:
m.save_pretrained("./mbins", from_pt=True)

In [ ]:
import torch

torch.save(m, "./mbins/merged_mamba.pt")

In [ ]:
torch.save(base_model, "./mbins/base_mamba.pt")

### Merge two PyTorch models
Suppose we have two models with same architecture
How can we combine them -- consider a simple weighed average at this time

In [ ]:
# https://github.com/ultralytics/yolov5/issues/12054
import torch
from torch import nn

sdA = m.state_dict()
sdB = base_model.state_dict()

# Merge the state dictionaries
sdC = {}
for key in sdA:
    sdC[key] = (sdA[key] + sdB[key]) / 2

weighed_model = MambaForCausalLM.from_pretrained("Q-bert/Mamba-130M")
weighed_model.load_state_dict(sdC)

# Save the merged model
torch.save(weighed_model.state_dict(), "./mbins/fed_mamba.pt")

In [ ]:
fed_model = MambaForCausalLM.from_pretrained("Q-bert/Mamba-130M")
fed_model.load_state_dict(torch.load("./mbins/fed_mamba.pt"))


output = fed_model.generate(input_ids, max_length=20, num_beams=5, no_repeat_ngram_size=2)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

### Create two adapters and merge adapters
Instead of model mering, let us merge adapters using Peft methods